In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_mypy

In [ ]:
import chess
import nbimporter
from typing import Any
from Exercise03AI import Exercise03AI

# Aufgabe 05: Minimax mit Alpha-Beta-Pruning

Dieses Notebook implementiert den Minimax-Algorithmus mit Alpha-Beta-Pruning (ohne Memoisierung). Hierzu wird die `minimax`-Funktion adaptiert.

In [ ]:
class Exercise05AI(Exercise03AI):
    """Chooses middle game moves using minimax algorithm and alpha-beta-pruning."""
    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)

## Alpha-Beta Pruning

Alpha-Beta Pruning ist eine Suchtechnik aus den späteren 1950er Jahren, bei der große Teile des Suchbaumes übersprungen werden können[9]. Gegenüber dem bisherigen Minimax werden hierfür die Parameter $\alpha$ und $\beta$ hinzugefügt, die angeben, welches Ergebnis die Spieler bei optimaler Spielweise erreichen können. Mithilfe dieser Parameter kann entschieden werden, welche Teile des Suchbaumes nicht untersucht werden müssen, da sie nicht zum Ergebnis beitragen[9]. Die folgenden Gleichungen beschreiben das Verhalten der neuen Funktion $\textrm{alphaBetaMinimax}$ gegenüber der bisherigen $\textrm{minimax}$-Funktion[2]. Hierbei ist $s$ der aktuelle Spielzustand (Board) und $d \in \mathbb{N}$ die gewählte Suchtiefe.

$$
\begin{align*}
&\alpha \le \textrm{minimax}(s,d) \le \beta &&\implies \textrm{alphaBetaMinimax}(s,d,\alpha,\beta) = \textrm{minimax}(s,d)\\
&\textrm{minimax}(s,d) < \alpha &&\implies \textrm{alphaBetaMinimax}(s,d,\alpha,\beta) \le \alpha\\
&\beta < \textrm{minimax}(s,d) &&\implies \beta \le \textrm{alphaBetaMinimax}(s,d,\alpha,\beta)\\
\end{align*}
$$

Mit dieser Spezifikation kann der Aufruf $\textrm{minimax}(s,d)$ durch den Aufruf $\textrm{alphaBetaMinimax}(s,d,-\textrm{LIMIT},\textrm{LIMIT})$ ersetzt werden. Hierbei ist $\textrm{LIMIT}$ die obere bzw. untere Grenze für alle möglichen Evaluierungen. In dieser Implementierung wurde das Limit als $99999$ bzw. $-99999$ gewählt. Für die Rückwärtskompatibilität wurde der Funktionsname $\textrm{minimax}$ in der folgenden Implementierung beibehalten und überschreibt die Funktion der Oberklasse.

![](./images/AlphaBetaPruning.jpg)

Das o.g. Bild aus [A2] zeigt das Überspringen oder Abschneiden von Teilbäumen, wenn dort keine bessere Beurteilung zu erwarten ist. In diesem Bild wird von links nach rechts und von unten nach oben ausgewertet. Die grau dargestellten Beurteilungen werden in der Praxis nicht berechnet und sind nur für ein besseres Verständnis enthalten.

## Minimax (aktualisiert)

Die rekursive `minimax`-Funktion betrachtet bei einem gegebenen Board alle möglichen Stellungen (Boards) nach einer gegebenen Anzahl von Halbzügen (Tiefe) und gibt die beste Evaluierung (`evaluate`-Funktion) für den aktuellen Spieler zurück. Hierfür bekommt die Funktion als Argumente das aktuelle Board (`board`), die Evaluierungstiefe in Halbzügen (`depth`), die Evaluierung des übergebenen Boards (`current_evaluation`), eine untere Grenze für die Evaluierung (`alpha`) und eine obere Grenze für die Evaluierung (`beta`). Zurückgegeben wird ein Tupel bestehend aus der besten Evaluierung und des Halbzuges, der auf den Pfad zu dieser Evaluierung führt.

Die folgende Implementierung des Minimax-Algorithmus ist zu großen Teilen mit der von `Exercise02AI` identisch. Als Neuerung wurde Alpha-Beta-Pruning hinzugefügt. Im Folgenden werden nur die Änderungen betrachtet:

1. Die `minimax`-Funktion bekommt zwei zusätzliche Parameter $\textrm{alpha}$ und $\textrm{beta}$ welche mithilfe von Standardwerten beim ersten Aufruf auf das positive bzw. negative Limit gesetzt werden.
2. Wenn der weiße Spieler am Zug ist (Maximierung) wird `maxEvaluation` auf den Wert $\textrm{alpha}$ gesetzt. Wie bisher wird nun jeder Zug evaluiert. Hierbei wird das Maximum der bisherigen Evaluationen jeweils als neue untere Grenze $\textrm{alpha}$ eingesetzt. Falls eine Evaluierung größer oder gleich $\textrm{beta}$ ist, wird die Suche abgebrochen (pruning) und diese Evaluierung zurückgegeben.
2. Falls der schwarze Spieler am Zug ist (Minimierung) wird `minEvaluation` auf den Wert $\textrm{beta}$ gesetzt. Wie bisher wird nun jeder Zug evaluiert. Hierbei wird das Minimum der bisherigen Evaluationen jeweils als neue obere Grenze $\textrm{beta}$ eingesetzt. Falls eine Evaluierung kleiner oder gleich $\textrm{alpha}$ ist, wird die Suche abgebrochen (pruning) und diese Evaluierung zurückgegeben.

Für einen Aufruf mit Weiß am Zug gilt also: 
$$\textrm{return_value} = \begin{cases}
\textrm{alpha} \quad \texttt{falls} \quad \textrm{evaluation} < \textrm{alpha} \quad \texttt{für alle} \quad \textrm{move} \in \textrm{board.legal_moves},\\
\ge \textrm{beta} \quad \texttt{falls} \quad \textrm{move} \in \textrm{board.legal_moves} \quad \texttt{mit} \quad \textrm{evaluation} \ge \textrm{beta} \quad \texttt{existiert,}\\
\textrm{evaluation} \quad \texttt{sonst}.
\end{cases}$$
Für einen Aufruf mit Schwarz am Zug gilt: 
$$\textrm{return_value} = \begin{cases}
\textrm{beta} \quad \texttt{falls} \quad \textrm{evaluation} > \textrm{beta} \quad \texttt{für alle} \quad \textrm{move} \in \textrm{board.legal_moves},\\
\le \textrm{alpha} \quad \texttt{falls} \quad move \in \textrm{board.legal_moves} \quad \texttt{mit} \quad \textrm{evaluation} \le \textrm{alpha} \quad \texttt{existiert,}\\
\textrm{evaluation} \quad \texttt{sonst}.
\end{cases}$$
Somit ist in beiden Fällen die oben gegebene formale Spezifikation für das Alpha-Beta-Pruning erfüllt und das Programm liefert dieselbe Auswertung wie die Implementierung in `Exercise02AI`.

In [ ]:
class Exercise05AI(Exercise05AI):  # type: ignore
    def minimax(
        self,
        board: chess.Board,
        depth: int,
        current_evaluation: int,
        alpha: int = -Exercise05AI.LIMIT,
        beta: int = Exercise05AI.LIMIT,
    ) -> tuple[int, chess.Move | None]:
        """Searches the best value with a given depth using the minimax algorithm."""
        self.stats[-1]["nodes"] += 1
        early_abort_evaluation = self.minimax_early_abort(board, depth, current_evaluation)
        if early_abort_evaluation is not None:
            return early_abort_evaluation, None

        best_move = None

        # White to play (positive numbers are good)
        if board.turn:
            maxEvaluation = alpha
            for move in board.legal_moves:
                new_evaluation = self.incremental_evaluate(board, current_evaluation, move)
                board.push(move)
                evaluation, _ = self.minimax(
                    board,
                    depth - 1,
                    new_evaluation,
                    maxEvaluation,
                    beta,
                )
                board.pop()
                if evaluation >= beta:
                    return evaluation, move
                if depth == self.DEPTH and evaluation > maxEvaluation:
                    best_move = move
                maxEvaluation = max(maxEvaluation, evaluation)
            return maxEvaluation, best_move

        # Black to play (negative numbers are good)
        else:
            minEvaluation = beta
            for move in board.legal_moves:
                new_evaluation = self.incremental_evaluate(board, current_evaluation, move)
                board.push(move)
                evaluation, _ = self.minimax(
                    board,
                    depth - 1,
                    new_evaluation,
                    alpha,
                    minEvaluation,
                )
                board.pop()
                if evaluation <= alpha:
                    return evaluation, move
                if depth == self.DEPTH and evaluation < minEvaluation:
                    best_move = move
                minEvaluation = min(minEvaluation, evaluation)
            return minEvaluation, best_move

## Debugging Bereich

Die folgenden Zellen enthalten Unit-Tests der oben implementierten Funktionen.

In [ ]:
from AIBaseClass import ChessAI
import Exercise02AI as Exercise02AI_

In [ ]:
# Create player and board
unit_test_player = Exercise05AI(player_name="Ex05AI", search_depth=3)
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 b - - 0 36")
board

Um das Alpha-Beta-Pruning zu testen, wird der Minimax-Unittest analog zur `Exercise03AI` aufgerufen. Während für den verwendeten Zug und die erwartete Evaluierung dieselben Werte verwendet werden, ist durch die `expected_nodes`-Metrik zu sehen, dass der Berechnungsaufwand sich von ursprünglich $14377$ auf $2788$ Knoten reduziert hat.

In [ ]:
# Use minimax test without memoization
Exercise02AI_.test_minimax(
    unit_test_player,
    board,
    current_evaluation=1240,
    expected_evaluation=325,
    expected_move="d4c3",
    expected_nodes=2788,
)